In [26]:
from bs4 import BeautifulSoup as bs
import requests

from pytube import YouTube
import time

import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import json
import os
from urllib.request import Request, urlopen



In [27]:
# initialize search strings here
qstrings = ["red", "green"]

#initialize request lists
base = "https://www.youtube.com/results?search_query="

rList = []
for i in qstrings:
  rList.append(requests.get(base+i))

soups=[]
for i in rList:
  soups.append(bs(i.text,'html.parser'))

allvids = []
for i in soups:
  allvids.append(i.findAll('a',attrs={'class':'yt-uix-tile-link'}))

flat_vids = []
for sublist in allvids:
    for item in sublist:
        flat_vids.append(item)

# final list of video links here
urllist = []
for v in flat_vids:
  tmp = 'https://www.youtube.com' + v['href']
  urllist.append(tmp)



def sublists(a, tolerance):
    result = []
    index = 0

    while index < len(a):
        curr = a[index]

        for i in range(index, len(a)):
            if a[i] == curr:
                end = i
            elif i - end > tolerance:
                break

        if index != end:
            result.append(a[index:end+1])
        index += end - index + 1

    return result

urllist = sublists(urllist, 6)
flattened = []
for sublist in urllist:
    for val in sublist:
        flattened.append(val)
urllist = flattened
urllist

['https://www.youtube.com/watch?v=zxG4PgMy1KU',
 'https://www.youtube.com/watch?v=trryp3FsrII',
 'https://www.youtube.com/watch?v=zxG4PgMy1KU']

In [28]:
# For ignoring SSL certificate errors

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE



In [29]:
video_dic = []
id = 1
urllist[1]
#YouTube(urllist[1]).player_config_args['player_response']['videoDetails']['lengthSeconds']
YouTube(urllist[1] + " -4")

RegexMatchError: regex pattern ("sts"\s*:\s*(\d+)) had zero matches

In [18]:
def fetch_video_details(url,id,description):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
#     prefix = "Praty test"
    video_details = {}

    video_details['ID'] = id
    for span in soup.findAll('span',attrs={'class': 'watch-title'}):
        video_details['TITLE'] = span.text.strip()

    for script in soup.findAll('script',attrs={'type': 'application/ld+json'}):
            channelDesctiption = json.loads(script.text.strip())
            video_details['CHANNEL_NAME'] = channelDesctiption['itemListElement'][0]['item']['name']

    for div in soup.findAll('div',attrs={'class': 'watch-view-count'}):
        video_details['NUMBER_OF_VIEWS'] = div.text.strip()

    for button in soup.findAll('button',attrs={'title': 'I like this'}):
        video_details['LIKES'] = button.text.strip()

    for button in soup.findAll('button',attrs={'title': 'I dislike this'}):
        video_details['DISLIKES'] = button.text.strip()

    for span in soup.findAll('span',attrs={'class': 'yt-subscription-button-subscriber-count-branded-horizontal yt-subscriber-count'}):
        video_details['NUMBER_OF_SUBSCRIPTIONS'] = span.text.strip()
    video_details['DESCRIPTION'] = description
    hashtags = []
    # for span in soup.findAll('span',attrs={'class': 'standalone-collection-badge-renderer-text'}):
    #     for a in span.findAll('a',attrs={'class': 'yt-uix-sessionlink'}):
    #         hashtags.append(a.text.strip())
    hashtags = soup.find("meta", {"name":"keywords"})['content']
    video_details['HASH_TAGS'] = hashtags
    
    
    # print(hashtags)
   
    return(video_details)

In [6]:
def download_video(url,prefix):
    urlForYt = url + " -4"
    yt = YouTube(urlForYt)
    videoList = yt.streams.filter(res='360p',mime_type='video/mp4',only_video=True).first()
    print(videoList)

    yt.streams.filter(res='360p',mime_type='video/mp4',only_video=True).first().download(filename_prefix=prefix)
    print(url + " downloaded")
    return(yt.description)

In [21]:
id = 1
for i in urllist:
    try:
        #url = i + " -4"
        url = i
        print(url)
        yt = YouTube(url)
        desc = yt.description
        print(desc)
        if 10 < yt.player_config_args['player_response']['videoDetails']['lengthSeconds'] < 600:
        #   fetch_video_details(i)
            prefix=str(id) + "_"
            print("==============downloading video ==========")
            desc = yt.description
            yt.streams.filter(res='360p',mime_type='video/mp4',only_video=True).first().download(filename_prefix=prefix)
            video_details = fetch_video_details(i,prefix,desc)
            print(video_details)
            time.sleep(15)

            id = id + 1
            video_dic.append(video_details)
    except:
        print("failed")
        time.sleep(5)
        pass


https://www.youtube.com/watch?v=zxG4PgMy1KU
failed
https://www.youtube.com/watch?v=trryp3FsrII
failed
https://www.youtube.com/watch?v=zxG4PgMy1KU
failed


In [ ]:
with open('data.json', 'w', encoding='utf8') as outfile:
    json.dump(video_dic, outfile, ensure_ascii=False,indent=4)